# Load Dataset

In [5]:
from tqdm.auto import tqdm

In [6]:
from datasets import load_dataset

df = load_dataset("ai4bharat/IndicHeadlineGeneration", "ml")

In [7]:
df = df["test"].select(range(500))
print(df[:1])

{'id': ['1'], 'input': ['ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മ

In [8]:
texts = df['input']
references = df['target']

# Compute Metrics

In [9]:
import evaluate

# Load metrics once
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(predictions, references, rouge_stemmer=True):
    """
    Computes ROUGE and BLEU scores for given predictions and references.
    
    Args:
        predictions (list): List of predicted texts.
        references (list): List of reference texts.
        rouge_stemmer (bool): Whether to use stemming for ROUGE.
    
    Returns:
        dict: ROUGE and BLEU scores rounded to four decimal places.
    """
    
    # Compute ROUGE scores
    rouge_result = rouge.compute(predictions=predictions, references=references, use_stemmer=rouge_stemmer)
    
    # Compute BLEU scores
    bleu_result = bleu.compute(
        predictions=predictions,
        references=[[r] for r in references]
    )
    
    return {
        "ROUGE-1": round(rouge_result['rouge1'], 4),
        "ROUGE-2": round(rouge_result['rouge2'], 4),
        "ROUGE-L": round(rouge_result['rougeL'], 4),
        "BLEU": round(bleu_result["bleu"], 4)
    }

2025-06-17 11:53:52.708628: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 11:53:52.725584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750157632.744731 3420619 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750157632.750889 3420619 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750157632.767419 3420619 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Print Results

In [10]:
def print_results(predictions, num_results = 5):
    for i in range(num_results):
        print(f"Input Text:\n{texts[i]}\n")
        print(f"Generated Summary:\n{predictions[i]}\n")
        print(f"Reference Summary:\n{references[i]}\n")
        print("="*80)

# Bart

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "ai4bharat/IndicBART"  # Malayalam IndicBART

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [21]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")


In [22]:
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator


def malayalam_to_devanagari(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'ml', 'hi') + ' </s> <2hi>'
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'ml', 'hi') + ' </s> <2hi>' for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

def devanagari_to_malayalam(input_text):
    # Check if input is a string or a list of strings
    if isinstance(input_text, str):
        return UnicodeIndicTransliterator.transliterate(input_text, 'hi', 'ml')
    elif isinstance(input_text, list):
        # If it's a list, apply the transformation to each string
        return [UnicodeIndicTransliterator.transliterate(text, 'hi', 'ml') for text in input_text]
    else:
        raise ValueError("Input must be a string or a list of strings.")

In [9]:
texts = texts[:16]

In [23]:
batch_size = 8  # smaller batch size to show progress in chunks
predictions = []

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]
    batch_texts = malayalam_to_devanagari(batch_texts)
    # inputs = tokenizer(batch_texts, max_length=512, truncation=True, padding=True, return_tensors='pt')
    # inputs = tokenizer("मैं [MASK] हूँ </s> <2hi>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids
    inputs = tokenizer(batch_texts, add_special_tokens=False, padding=True, return_tensors='pt')

    # summary_ids = model.generate(
    #     inputs['input_ids'], attention_mask=inputs['attention_mask'],
    #     max_length=128, num_beams=4, early_stopping=True
    # )

    summary_ids = model.generate(
        inputs['input_ids'], use_cache=True, num_beams=4, max_length=128,
        early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id,
        eos_token_id=eos_id,
        decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>")
        )

    batch_preds = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    predictions.extend(batch_preds)

predictions = devanagari_to_malayalam(predictions)

Generating summaries:   0%|          | 0/63 [00:00<?, ?it/s]

In [24]:
compute_metrics(predictions, references)

{'ROUGE-1': np.float64(0.0504),
 'ROUGE-2': np.float64(0.0107),
 'ROUGE-L': np.float64(0.05),
 'BLEU': 0.0222}

In [25]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# Qwen

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "Qwen/Qwen2-1.5B-Instruct"


device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", trust_remote_code=True).to(device)

torch.cuda.empty_cache()

In [28]:
predictions=[]

for text in tqdm(texts, desc="Generating summaries"):
  try:
    prompt = f"""
      Generate a short and clear news headline in Malayalam based on the following text.

      Text to generate headline for: {text}

      The headline should capture the main idea of the text without copying it word-for-word.
      Return the headline in complete Malayalam.
      You are not to return the thought process or any non Malayalam texts.
      """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    with torch.no_grad():
      generated_ids = model.generate(
          model_inputs.input_ids,
          max_new_tokens=512
      )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # if "Summary:" in response:
    #   summary_clean = response.split("Summary:")[-1].strip()
    # else:
    #   summary_clean = response.strip()

    predictions.append(response)

    torch.cuda.empty_cache()

  except torch.cuda.OutOfMemoryError:
    print("⚠️ CUDA OOM – Skipping this input.")
    torch.cuda.empty_cache()
    predictions.append("[OOM ERROR – Skipped]")

Generating summaries:   0%|          | 0/500 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [29]:
compute_metrics(predictions, references)


{'ROUGE-1': np.float64(0.0122),
 'ROUGE-2': np.float64(0.002),
 'ROUGE-L': np.float64(0.0125),
 'BLEU': 0.0236}

In [30]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# Gemma



In [31]:
from huggingface_hub import login
TOKEN=None
login(token=TOKEN)

In [33]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch

model_id = "google/gemma-3-4b-it"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = Gemma3ForConditionalGeneration.from_pretrained(model_id).to(device).eval()

processor = AutoProcessor.from_pretrained(model_id, use_fast=True)
torch.set_float32_matmul_precision('high')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
predictions = []
torch.cuda.empty_cache()

for text in tqdm(texts, desc="Generating summaries"):
  try:
    prompt = f"""
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""

    messages = [
        [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a helpful assistant that generates news headlines."},]
            },
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt},]
            },
        ],
    ]

    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
      generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
      generation = generation[0][input_len:]

    # Slice off input tokens from generated tokens to get only new tokens
    # generated_only_ids = [
    #     gen_ids[len(input_ids[idx]):] for idx, gen_ids in enumerate(generated_ids)
    # ]

    decoded = processor.decode(generation, skip_special_tokens=True)
    predictions.append(decoded)

    torch.cuda.empty_cache()

  except torch.cuda.OutOfMemoryError:
    print("⚠️ CUDA OOM – Skipping this input.")
    torch.cuda.empty_cache()
    predictions.append("[OOM ERROR – Skipped]")

Generating summaries:   0%|          | 0/500 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not va

In [35]:
import re

predictions = [re.sub(r"(?i)^\s*Headline:\s*", "", p) for p in predictions]

In [36]:
compute_metrics(predictions, references)

{'ROUGE-1': np.float64(0.024),
 'ROUGE-2': np.float64(0.006),
 'ROUGE-L': np.float64(0.0242),
 'BLEU': 0.0205}

In [ ]:
print_results(predictions)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

: 

## By-T5-Small

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load ByT5 and Flan-T5
byt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/byt5-small").to(device)
byt5_tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

In [13]:
def generate_summary(text, model, tokenizer):
    prompt = f"""
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            num_beams=2,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()


In [16]:
byt5_preds = []

for text in tqdm(texts, desc="ByT5-Small"):
    generated_summary = generate_summary(text, byt5_model, byt5_tokenizer)
    byt5_preds.append(generated_summary)

# Save outputs
df['byt5_summary'] = byt5_preds

ByT5-Small:   0%|          | 0/500 [00:00<?, ?it/s]

TypeError: 'Dataset' object does not support item assignment

In [17]:
import re

# Remove any English word followed by a colon and text inside parentheses
byt5_preds = [re.sub(r"(?i)^\s*\w+:\s*|\s*\([^)]*\)|:\s*", "", p) for p in byt5_preds]


In [18]:
compute_metrics(byt5_preds, references)

{'ROUGE-1': np.float64(0.0023),
 'ROUGE-2': np.float64(0.0),
 'ROUGE-L': np.float64(0.0023),
 'BLEU': 0.0088}

In [19]:
print_results(byt5_preds)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

# **mT5**


In [20]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/umt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/umt5-base")

In [21]:
import torch
torch.cuda.empty_cache()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device);

In [22]:
prompt_template = """
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""



predictions = []
batch_size = 8  # Adjust depending on GPU memory

for i in tqdm(range(0, len(texts), batch_size), desc="Generating summaries"):
    batch_texts = texts[i:i+batch_size]

    prompts = [prompt_template.format(text=text) for text in batch_texts]

    # Tokenize inputs
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    
    ######################################################
    inputs.pop("token_type_ids", None) # used only with umt
    ######################################################

    # Generate summaries
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            early_stopping=True,
            num_beams=4,
            length_penalty=1.0,
            no_repeat_ngram_size=3,
        )

    # Decode outputs
    decoded_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    decoded_batch = [d.replace("summary:", "").strip() for d in decoded_batch]
    predictions.extend(decoded_batch)

# Now predictions has your model summaries for all texts

Generating summaries:   0%|          | 0/63 [00:00<?, ?it/s]

In [23]:
compute_metrics(predictions,references)

{'ROUGE-1': np.float64(0.0116),
 'ROUGE-2': np.float64(0.002),
 'ROUGE-L': np.float64(0.0115),
 'BLEU': 0.0054}

In [ ]:
print_results(predictions, 20)

Input Text:
ദില്ലി: ദില്ലിയില് പെണ്കുട്ടിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്ന് പോലീസ് കണ്ടെത്തിയതിനു പിന്നാലെ കഴിഞ്ഞ 12 മണിക്കൂറിനിടെ ദില്ലിയില് വീണ്ടും റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ടത് സമാനമായ 4 കേസുകള്.ഇതില് രണ്ടു പേര് പോലീസില് കേസ് രജിസ്റ്റര് ചെയ്തിട്ടുണ്ട്.ഉറങ്ങിയെഴുന്നേറ്റപ്പോള് മൂന്നിഞ്ച് നീളത്തില് മുടി നഷ്ടപ്പെട്ടതായി പരാതി നല്കിയ സ്ത്രീകളിലൊരാള് പോലീസിനെ അറിയിച്ചു.സംഭവത്തില് പോലീസ് അന്വേഷണമാരംഭിച്ചിട്ടുണ്ട്.എന്നാല് ഇതിനു പിന്നില് യുക്തപരമായ ഒരു ന്യായീകരണം നല്കാന് ഇപ്പോള് കഴിയില്ലെന്നും പോലീസ് പറയുന്നു.ദില്ലിയില് ഉറങ്ങിക്കിടന്ന 14 കാരിയുടെ മുടി മുറിച്ചത് സഹോദരന്മാരാണെന്നാണ് പോലീസ് കണ്ടെത്തിയത്.ഇതോടെ ദില്ലിയില് ഇത്തരത്തില് റിപ്പോര്ട്ട് ചെയ്യപ്പെട്ട ആദ്യ കേസിന്റെ ചുരുളഴിയുകയും ചെയ്തു.സംഭവത്തിനു പിന്നില് അസ്വാഭാവികത ഒന്നും ഇല്ലെന്നാണ് പോലീസ് പറയുന്നത്.പ്രേതത്തിന്റെ പേരും പറഞ്ഞ് ജനങ്ങള്ക്കിടയില് ഭീതി പരത്തരുതെന്നും പോലീസ് അറിയിച്ചു.എന്നാല് ഉറങ്ങിക്കിടക്കുന്ന സ്ത്രീകളുടെ മുടി പിറ്റേന്ന് ഉറങ്ങിയെഴുന്നേല്ക്കുമ്പോള് മുറിക്കപ്പെട്ട രീതിയില് കാണപ്പെടുന്ന അസാധാരണ സംഭവം ഇന്ത്യയിലെ മറ്റു സ്ഥലങ്ങള

: 

# Deepseek

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "deepseek-ai/deepseek-moe-16b-chat"

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
from types import MethodType

# Override `prepare_inputs_for_generation` to fix get_max_length -> get_seq_length
def patched_prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, **kwargs):
    if past_key_values:
        input_ids = input_ids[:, -1:]
    return {
        "input_ids": input_ids,
        "past_key_values": past_key_values,
        "use_cache": kwargs.get("use_cache", True),
        "attention_mask": attention_mask,
    }

# Attach the patched method to your model
model.prepare_inputs_for_generation = MethodType(patched_prepare_inputs_for_generation, model)


In [ ]:
# Prompt template
prompt_template = """
Generate a **clear and concise news headline in Malayalam only** based on the following text.

Text (Malayalam): {text}

Important:
- The output must be **only a headline in Malayalam**.
- Do **not** use any other language or script.
- Do **not** include any extra commentary or formatting.
- Do **not** copy the text word-for-word.
- Start your output with: Headline:

Example (do not include this in your output):
Headline: ഇന്ത്യയിൽ പുതിയ ശാസ്ത്രീയ കണ്ടെത്തൽ
"""

# Inference loop
predictions =[]

for text in tqdm(texts[:50], desc="Generating summaries"):
    prompt = prompt_template.format(text=text).strip()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the generated part (after the prompt)
    generated = output_text[len(prompt):].strip()
    generated = generated.replace("summary:", "").strip()
    
    predictions.append(generated)

    torch.cuda.empty_cache()


Generating summaries:   0%|          | 0/50 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['t

In [19]:
compute_metrics(predictions,references[:50])

{'ROUGE-1': np.float64(0.0),
 'ROUGE-2': np.float64(0.0),
 'ROUGE-L': np.float64(0.0),
 'BLEU': 0.0064}

In [ ]:
print_results(predictions)

Input Text:
ഇലക്ട്രോണിക്‌സിലും വിവരസാങ്കേതികവിദ്യയിലുമുള്ള ഉഭയകക്ഷിസഹകരണം പ്രോല്‍സാഹിപ്പിക്കാനുള്ള ഇന്ത്യ-അംഗോള ധാരണാപത്രത്തെക്കുറിച്ചു പ്രധാനമന്ത്രി ശ്രീ. നരേന്ദ്ര മോദിയുടെ അധ്യക്ഷതയില്‍ ചേര്‍ന്ന കേന്ദ്ര മന്ത്രിസഭാ യോഗം മുന്‍പാകെ വിശദീകരിക്കപ്പെട്ടു. ഇ-ഭരണം, ഐ.ടി.വിദ്യാഭ്യാസത്തിനുള്ള മനുഷ്യവിഭവശേഷി വികസനം, വിവരസാങ്കേതികവിദ്യാ സുരക്ഷ, ഇലക്ട്രോണിക് ഉല്‍പന്ന നിര്‍മാണം, സോഫ്റ്റ്‌വെയര്‍ വ്യവസായം, ടെലിമെഡിസിന്‍ തുടങ്ങിയ മേഖലകളില്‍ കൂടുതല്‍ സഹകരണം ഉറപ്പാക്കാന്‍ ഉദ്ദേശിച്ചുള്ളതാണു ധാരണാപത്രം. പശ്ചാത്തലം: ഉഭയകക്ഷി, മേഖലാതല സഹകരണത്തിനായുള്ള ചട്ടക്കൂടിന്റെ അടിസ്ഥാനത്തില്‍ വിവരസാങ്കേതികവിദ്യ, ആശയവിനിമയ സാങ്കേതികവിദ്യ എന്നീ മേഖലകളില്‍ രാജ്യാന്തര സഹകരണം പ്രോല്‍സാഹിപ്പിക്കുന്നതിന് ഇലക്ട്രോണിക്‌സ്, വിവരസാങ്കേതികവിദ്യ വകുപ്പു മന്ത്രാലയ(എം.ഇ.ഐ.ടി.വൈ.)ത്തെയാണു ചുമതലപ്പെടുത്തിയിരിക്കുന്നത്. വിവരസാങ്കേതികവിദ്യയില്‍ അധിഷ്ഠിതമായ സഹകരണം ഉറപ്പാക്കുന്നതിന് മറ്റു രാജ്യങ്ങളിലെ ബന്ധപ്പെട്ട ഏജന്‍സികളുമായി എം.ഇ.ഐ.ടി.വൈ. കരാറുകളും ധാരണാപത്രങ്ങളും ഒപ്പുവെച്ചിട്ടുണ്ട്. ഡിജിറ്റല്‍ ഇന്ത്യ, മെയ്ക്ക് ഇന്‍ ഇന്ത്യ തുടങ

: 